# Upload Terra model test
```
Andrew Davidson
aedavids@ucsc.edu
2/3/22
```


In [1]:
# Import the low-level FISS api library as `fapi` and other packages we will use throughout the tutorial
import firecloud.api as fapi
import urllib.request
import json
import sys
import pandas as pd
from pandas import DataFrame
import time
import os
import dumper # use to debug fissfc

In [2]:
billing_project = 'test-aedavids-proj'
my_email = 'aedavids@ucsc.edu'

In [3]:
# https://firebase.google.com/docs/auth/admin/verify-id-tokens#retrieve_id_tokens_on_clients
! gcloud auth login --update-adc

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=l1sOVs9tsopZNldcRuLvt5mwyEbGGw&access_type=offline&code_challenge=Yh1vRfiojKCHIlNRseP7-ovrlmz8VqlfzwZgBZCI-r4&code_challenge_method=S256


Application default credentials (ADC) were updated.

You are now logged in as [aedavids@ucsc.edu].
Your current project is [test-aedavids-proj].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [4]:
!pwd

/Users/andrewdavidson/googleUCSC/kimLab/extraCellularRNA/terra/jupyterNotebooks


In [5]:
#root = "terraDataModels-hack/test-aedavids-proj/TCGA"
# workspacesWithMissingQuantfiles = ['TCGA_CHOL_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
#                                      'TCGA_PAAD_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
#                                      'TCGA_SARC_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
#                                     ]
rootDir = "../../../terraDataModels/test-aedavids-proj/TCGA"

In [6]:
# find work spaces that have new set defintions
# assumes these files have not been commited yet

# we can not run this from notebook. 
# fatal: ../../../terraDataModels/test-aedavids-proj/TCGA: 
# '../../../terraDataModels/test-aedavids-proj/TCGA' is outside repository at 
# '/Users/andrewdavidson/googleUCSC/kimLab/extraCellularRNA'
# !git status --porcelain $rootDir |grep set_entity.tsv | cut -d ' ' -f 3

In [7]:
#listOfWorkSpacePath = rootDir + "/" + "listOfWorkSpaces.txt" 
listOfWorkSpacePath = rootDir + "/" + "listOfFailedSamples.txt"
workspaceDF = pd.read_csv( listOfWorkSpacePath , names=["workspace"])
workSpaceNamesList = workspaceDF.loc[:, "workspace"].to_list()
workSpaceNamesList

['TCGA_BRCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_COAD_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_LUAD_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_PCPG_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_PRAD_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_READ_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_STAD_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_THCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_UCEC_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab']

In [8]:
ERROR_CODE = 1
OK_CODE = 0

def uploadSetModel(billing_project, entityPath ):
    '''
    Upload data in the tsv to the workspace as a table
    By default, the upload_entities_tsv command expects one of the five entity types in the stadard 
    genomic data model (which include the participant, sample, pair, and sets of these entitie
    If your table is not part of the standard genomic data model, you need the model='flexible' parameter. 
    https://github.com/broadinstitute/fiss/blob/master/firecloud/api.py
    https://docs.python-requests.org/en/v0.6.2/api/
    '''

    response = None
    error = None
    try:
        response = fapi.upload_entities_tsv(billing_project, workspace, entityPath, model='flexible')
    except Exception as error:        
        print("ERROR {} failed".format(entityPath))
        print("exception\n{}".format(error))
        return ERROR_CODE
    
    statusCode = getattr(response, "status_code")        
    if statusCode != 200:
        # debuging
        print( "ERROR statusCode: {} \n{}".format(statusCode, entityPath) )
        print(dumper.dump(response)) 
        return ERROR_CODE
    else:
        print("{} completed status = 200\n".format(entityPath))
        
    return OK_CODE

In [9]:
for workspace in workSpaceNamesList:
    print()
    print(workspace)
    setEntityPath = rootDir + "/" + workspace + "/sample_set_entity.tsv" 
    exitStatus = uploadSetModel(billing_project, setEntityPath )
    if exitStatus == OK_CODE:
        setMembershipPath = rootDir + "/" + workspace + "/sample_set_membership.tsv"    
        exitStatus = uploadSetModel(billing_project, setMembershipPath )


TCGA_BRCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab
../../../terraDataModels/test-aedavids-proj/TCGA/TCGA_BRCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab/sample_set_entity.tsv completed status = 200

ERROR ../../../terraDataModels/test-aedavids-proj/TCGA/TCGA_BRCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab/sample_set_membership.tsv failed
exception
HTTPSConnectionPool(host='api.firecloud.org', port=443): Read timed out. (read timeout=120)

TCGA_COAD_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab
../../../terraDataModels/test-aedavids-proj/TCGA/TCGA_COAD_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab/sample_set_entity.tsv completed status = 200

../../../terraDataModels/test-aedavids-proj/TCGA/TCGA_COAD_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab/sample_set_membership.tsv completed status = 200


TCGA_LUAD_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab
../../../terraDataModels/test-aedavids-proj/TCGA/TCGA_LUAD_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab/sample_set_entity.tsv completed statu

In [10]:
# re run time
setMembershipPath = rootDir + "/" + 'TCGA_BRCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab' + "/sample_set_membership.tsv"    
exitStatus = uploadSetModel(billing_project, setMembershipPath )

ERROR statusCode: 400 
../../../terraDataModels/test-aedavids-proj/TCGA/TCGA_BRCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab/sample_set_membership.tsv
<Response at 7f8f91c91150: <Response [400]>> 
  _content: <str at 0x7f8f88af0000>: 'b\'{"causes":[{"causes":[],"message":"sample BRCA-BH-A0DL-NT not found","source":"rawls","stackTrace":[]},{"causes":[],"message":"sample BRCA-E2-A14P-TP not found","source":"rawls","stackTrace":[]},{"causes":[],"message":"sample BRCA-AO-A12H-TP not found","source":"rawls","stackTrace":[]},{"causes":[],"message":"sample BRCA-C8-A1HG-NB not found","source":"rawls","stackTrace":[]},{"causes":[],"message":"sample BRCA-AO-A03T-NB not found","source":"rawls","stackTrace":[]},{"causes":[],"message":"sample BRCA-E9-A1RE-TP not found","source":"rawls","stackTrace":[]},{"causes":[],"message":"sample BRCA-EW-A1OV-TP not found","source":"rawls","stackTrace":[]},{"causes":[],"message":"sample BRCA-EW-A1P7-TP not found","source":"rawls","stackTrace":[]},{"causes":[],"

# update GTEx

In [ ]:
rootDir       = "../../../terraDataModels/test-aedavids-proj" 
workspace     = "AnVIL_GTEx_V8_hg38_edu_ucsc_kim_lab"   
setEntityPath = rootDir + "/" + workspace + "/sample_set_entity.tsv"

exitStatus = uploadSetModel(billing_project, setEntityPath )
if exitStatus == OK_CODE:
setMembershipPath = rootDir + "/" + workspace + "/sample_set_membership.tsv"    
exitStatus = uploadSetModel(billing_project, setMembershipPath )

